# Pelabelan Data Menggunakan Pretrained Model
Menggunakan Transformer dengan Model Indonesian RoBERTa Base Sentiment Classifier

## Import Library

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline

## Import Data dari Proses Sebelumnya

In [2]:
df = pd.read_csv('data/tweets_clean.csv')

## Mempersiapkan Model

In [ ]:
pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

## Polarity Scoring and Labelling

In [ ]:
df2 = pd.DataFrame(columns = ['tweet', 'sentimen', 'skor'])
for i in tqdm(df['tweet'],desc='Scoring and Labelling..'):
    sentiment = nlp(i)[0]['label']
    score = nlp(i)[0]['score']
    df2 = pd.concat([df2, pd.DataFrame([{"tweet" : i, "sentimen" : sentiment, "skor" : score}])])
    
def change_languange(text):
    if text == "neutral":
        return "Netral"
    if text == "positive":
        return "Positif"
    if text == "negative":
        return "Negatif"

df2['sentimen'] = df2['sentimen'].apply(change_languange)

In [ ]:
df2

## Visualization

### Total tweet positif, negatif, atau netral

In [ ]:
print("Positif :",len(df2[df2.sentimen=="Positif"]), " tweet")
print("Netral :",len(df2[df2.sentimen=="Netral"]), " tweet")
print("Negatif :",len(df2[df2.sentimen=="Negatif"]), " tweet")

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 30):
#      with open('data/daftar_kata.txt', 'w') as f:
#         print(df2['tweet'].str.split(expand=True).stack().value_counts(), file=f)

### Pie chart dari data labelling

In [ ]:
y = np.array([len(df2[df2.sentimen=="Positif"]),  len(df2[df2.sentimen=="Netral"]), len(df2[df2.sentimen=="Negatif"])])
mylabels = ['Positif', 'Netral', 'Negatif']
mycolors = ['lightblue', 'lightgreen', 'orange']
myexplode = [0, 0.2, 0]

plt.rcParams['text.color'] = 'black'
plt.pie(y, colors=mycolors, labels = mylabels, explode = myexplode, shadow=True, autopct='%1.0f%%')
plt.show()

### Wordcloud semua data, positif, netral, dan negatif

In [ ]:
mask = np.array(Image.open("data/cloud.png"))

def plot_cloud(title, text):
    wc = WordCloud(scale=3,max_words=100,font_path="data/font/GothamMedium.ttf",background_color='white',
                   mask=mask,contour_color='black',contour_width=1).generate(str(" ".join(text)))
    # Set figure size
    plt.figure(figsize=(40,30))
    # Insert image wordcloud
    plt.imshow(wc) 
    # No axis details
    plt.axis("off")
    # Add Title
    plt.title(title)
    # Display image
    plt.show()

In [ ]:
text = df2['tweet'].astype('U')

plot_cloud("Semua Data", text)

In [ ]:
text_pos = df2[df2.sentimen=="Positif"].tweet.astype('U')

plot_cloud("Data Positif", text_pos)

In [ ]:
text_net = df2[df2.sentimen=="Netral"].tweet.astype('U')

plot_cloud("Data Netral", text_net)

In [ ]:
text_neg = df2[df2.sentimen=="Negatif"].tweet.astype('U')

plot_cloud("Data Negatif", text_neg)

## Export data

### Ekspor data per kata positif, negatif, atau netral

In [ ]:
df2_positif = df2[df2.sentimen=="Positif"]
df2_netral = df2[df2.sentimen=="Netral"]
df2_negatif = df2[df2.sentimen=="Negatif"]

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 30):
    with open('data/temp/daftar_kata_all_roberta.txt', 'w') as f:
        print(df2['tweet'].str.split(expand=True).stack().value_counts(), file=f)
    with open('data/temp/daftar_kata_positif_roberta.txt', 'w') as f:
        print(df2_positif['tweet'].str.split(expand=True).stack().value_counts(), file=f)
    with open('data/temp/daftar_kata_netral_roberta.txt', 'w') as f:
        print(df2_netral['tweet'].str.split(expand=True).stack().value_counts(), file=f)
    with open('data/temp/daftar_kata_negatif_roberta.txt', 'w') as f:
        print(df2_negatif['tweet'].str.split(expand=True).stack().value_counts(), file=f)

### Ekspor data labelling untokenized

In [ ]:
df2.to_csv("data/tweets_labelled_roberta.csv", index=False)

### Ekspor data labelling tokenized

In [ ]:
nltk.download('punkt')
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df2['tweet'] = df2['tweet'].apply(word_tokenize_wrapper)
df2.to_csv("data/tweets_labelled_tokenized_roberta.csv", index=False)